# <center>Amortization Schedule using Pandas</center>

## Formula for Annuity : 

$$A = P\left(i+ \frac{i}{(1+i)^n-1}\right)$$
Where:

    A = Periodic Payment amount
    P = Amount of principal, net of initial payments, meaning *subtract any down-payments
    i = Periodic interest rate
    n = Total number of payments


### Building Amortization Schedule with Pandas: 
    
####  INPUT :
 - Annual Interest Rate 
 - Loan Amount 
 - Loan Term (in years)
 - Number of Payments Per year

####  OUTPUT :
 - 6 x 9 Pandas Dataframe for Payments at various rates and term 
 - Pandas DataFrame for Amortization Payment Schedule as per User Input <br>
 - Pandas DataFrame for summary of Total Interest,Total Principal and Total Paid <br>

In [195]:
#Libraries needed for this project
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [196]:
#Input Validation for Principal, annual rate , loan term and no of payments

class InputValidation:
    """
    This class validates the inputs (Principal, annual rate, term in years and number of payments per year)
    Principal(principal) : Validates if input Loan amount is a positive integer
    Annual Rate(annual_interest_rate) : Validates if the Annual rate is >0
    Term in Years(per): Validates if term is a positive finite integer
    No of Payments(nper):Validates if nper is a positive finite integer
    """
    def __init__(self,inputvar):
        self.inputvar = inputvar
    
    #Method for checking if the input variable is a positive integer value (Eg. Principal)
    def check_positive_integer(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar>0:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar

    #Method for checking if the input variable is a positive value (Eg. Annual interest rate)
    def check_positive_float(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar>0:
                    break
                else:
                    print(f"\n{self.inputvar} should be a positive value greater than 0!")
            except:
                print("\nNot a valid Option!! Try again..")
        return tempvar
        
    #Method for checking if the input variable is a finite term (Eg.Loan term and No. of payments per year)
    def finiteterm_check(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar>0 and tempvar<100:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value less than 100!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar
       

In [197]:
# Function to generate matrix of various term vs rate of interest
def generate_matrix(principal,paymentfreq,initial_rate,ratestep,period,periodstep):
    """
    This function generates the payment matrix at various rates and terms
    """
    # Generates a matrix of 6 x 9 matrix with payments at various 
    
    # Declaring Row iterator
    termrows = [i for i in range(period,period+30,periodstep)]
    
    # Declaring Column Iterator
    ratecolumn =[(round(j*100,3)) for j in np.arange(initial_rate,initial_rate+ 4.5/100,ratestep)]
    
    # Naming axis and index
    mymatrix = pd.DataFrame(columns =ratecolumn ,index=termrows).rename_axis('Annual rate',axis=1)    
    mymatrix['Period']=termrows
    mymatrix.set_index('Period',inplace=True)
    
    # Generating Payment matrix for generated list of period and rates
    for i in termrows:
        for j in ratecolumn:
            mymatrix.at[i,j] = -np.pmt(float(j/100)/paymentfreq,i*paymentfreq,principal)
    
    return (mymatrix)

In [198]:
def generate_payment_schedule(principal,annual_interest_rate,per,nper):
    """
    This function generates the amortization schedule
    """
    # Declaration of Variables
    periodic_interest_rate = annual_interest_rate/nper
    no_of_payments = nper*per
    
    # Defining the Structure of DataFrame
    columnnames =['Period','Opening Balance','Payment','Interest Expense','Repayment of Principal','Closing Balance']

    # Filling Static Columns & Index
    period=[i for i in range(1,no_of_payments+1)]
    
    # Formatting the DataFrame
    pd.options.display.float_format = '${:,.2f}'.format
 
    # Initialization of the DataFrame
    mymatrix = pd.DataFrame(columns =columnnames,index=period)
    
    # Calculations 
    mymatrix.at[1,'Opening Balance']=principal
    mymatrix['Period']=period
    mymatrix.set_index('Period',inplace=True)
    mymatrix['Payment']=-np.pmt(periodic_interest_rate,no_of_payments,principal)
    mymatrix['Interest Expense']=-np.ipmt(periodic_interest_rate,mymatrix.index, no_of_payments,principal)
    mymatrix['Repayment of Principal']= -np.ppmt(periodic_interest_rate,mymatrix.index,no_of_payments,principal)
    

    #Calculation of dynamic part of Amortization Schedule
    for i in period:
        if i>1:
            mymatrix['Closing Balance']= mymatrix['Opening Balance']-mymatrix['Repayment of Principal']
            mymatrix.at[i,'Opening Balance']=mymatrix.at[i-1,'Closing Balance']
        if mymatrix.at[i,'Opening Balance']-mymatrix.at[i,'Repayment of Principal']<0.1:
            mymatrix.at[i,'Closing Balance']=0


    mymatrix.at[1,'Opening Balance']="$"+str(principal)
    return (mymatrix)

In [199]:
def generate_summary(principal,initial_rate,per,nper):
    """
    This function generates summary for amortization schedule
    """
    mysum = generate_payment_schedule(principal,initial_rate,per,nper)
    total_interest = mysum["Interest Expense"].sum().round(2)
    pmt = -np.pmt(initial_rate/nper,per*nper,principal)
    total=total_interest+principal
    mymatrix = pd.DataFrame([pmt,total_interest,total],columns=[""],index=["Payment per period","Total Interest","Total Payments"])
    return mymatrix

In [201]:
def amortization_table():
    """ Calculate the loan amortization schedule given the loan details

     Arguments:
        annual_interest_rate: The annual interest rate for this loan
        per: Number of years for the loan
        nper: Number of payments in a year
        principal: Amount borrowed

    Returns:
        matrix : Returns a 6 x 9 matrix of payments at various rates and term
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    # Taking User Input     
    principalval = InputValidation("Principal")
    principal= principalval.check_positive_integer()

    rateval = InputValidation("Annual Interest Rate")
    annual_interest_rate=rateval.check_positive_float()/100
    
    pervalidation = InputValidation("Loan Term in years")
    per = int(pervalidation.finiteterm_check())

    npervalidation = InputValidation("No of Payments per year")
    nper = int(npervalidation.finiteterm_check())
    
    # Formatting the DataFrame
    pd.options.display.float_format = '${:,.2f}'.format
    

    # Payment at various rates vs term
    matrix = generate_matrix(principal,nper,annual_interest_rate,0.005,per,5)

    # Amortization Schedule
    schedule = generate_payment_schedule(principal,annual_interest_rate,per,nper)
    summary= generate_summary(principal,annual_interest_rate,per,nper)
    
    print("\n               \033[1m   Payments at various rate vs term \033[1m ")
    display(matrix)
    
    print("\n \033[1m Based on the information you entered, your payment is ${} for {} years with a rate of {}%\033[1m".format(summary.at["Payment per period",""].round(2),per,annual_interest_rate*100))

    print("  \n                      \033[1m Payment Schedule\033[1m ")
    display(schedule)
    print("  \n   \033[1m Payment Summary\033[1m ")
    display(summary)
    

In [ ]:
amortization_table()